In [1]:
from app.backend.core.file_processor import FileProcessor
from app.backend.core.analyzer import Analyzer

processor = FileProcessor()
analyzer = await Analyzer.create()

2025-07-16 20:43:56 | app.backend.services.groq | DEBUG | Retrieving Groq API key from Azure Key Vault...
2025-07-16 20:43:58 | app.backend.services.groq | DEBUG | Groq API key retrieved.


In [10]:
job = """
TestCorp LLC. is at the forefront of digital reinvention, helping clients reimagine how they serve their connected customers and operate enterprises. We’re looking for an experienced artificial intelligence engineer to join the revolution, using deep learning, neuro-linguistic programming (NLP), computer vision, chatbots, and robotics to help us improve various business outcomes and drive innovation. The engineer will join a multidisciplinary team helping to shape our AI strategy and showcasing the potential for AI through early-stage solutions. This is an excellent opportunity to take advantage of emerging trends and technologies and make a real-world difference.
Objectives of this role
	• Manage and direct processes and R&D (research and development) to meet the needs of our AI strategy
	• Understand company and client challenges and how integrating AI capabilities can help lead to solutions
	• Lead cross-functional teams in identifying and prioritizing key areas of a partner’s business where AI solutions can drive significant business benefit
	• Analyze and explain AI and machine learning (ML) solutions while setting and maintaining high ethical standards
Responsibilities
	• Advise C-suite executives and business leaders on a broad range of technology, strategy, and policy issues associated with AI
	• Work on functional design, process design (including scenario design, flow mapping), prototyping, testing, training, and defining support procedures, in collaboration with an advanced engineering team and executive leadership
	• Articulate and document the solutions architecture and lessons learned for each exploration and accelerated incubation
	• Manage a team in conducting assessments of the AI and automation market and competitor landscape
	• Serve as liaison between stakeholders and project teams, delivering feedback and enabling team members to make necessary changes in product performance or presentation
Required skills and qualifications
	• Two or more years of experience in applying AI to practical and comprehensive technology solutions
	• Experience with ML, deep learning, TensorFlow, Python, NLP
	• Experience in program leadership, governance, and change enablement
	• Knowledge of basic algorithms, object-oriented and functional design principles, and best-practice patterns
	• Experience in REST API development, NoSQL database design, and RDBMS design and optimizations
Preferred skills and qualifications
	• Bachelor’s or master's degree in computer science or related field
	• Experience with innovation accelerators
	• Experience with cloud environments

"""

In [11]:
job_response = await analyzer.analyze_job(job)

2025-07-16 20:46:23 | app.backend.services.groq | INFO | Running structured query with function calling using ainvoke...
2025-07-16 20:46:24 | app.backend.services.groq | DEBUG | Beautified JSON args:
{
    "certificate": [],
    "domain": "Artificial Intelligence",
    "educational_requirements": [{
        "fields": ["Computer Science"],
        "level": "Bachelor's"
    }, {
        "fields": ["Computer Science"],
        "level": "Master's"
    }],
    "employment_type": "",
    "experience": ["2+ years in applying AI to practical and comprehensive technology solutions", "Experience in program leadership, governance, and change enablement", "Experience with ML, deep learning, TensorFlow, Python, NLP", "Experience in REST API development, NoSQL database design, and RDBMS design and optimizations"],
    "ideal_candidate_summary": "",
    "job_level": "",
    "job_title": "Artificial Intelligence Engineer",
    "location_requirement": "",
    "responsibilities": ["Manage and direct pr

In [6]:
resume = await processor.extract_from_file("data/ZacharyBiery_CV_2024.pdf")
candidate_response = await analyzer.analyze_candidate(resume)

2025-07-16 20:45:12 | app.backend.core.file_processor | INFO | Extracting text from local file: data/ZacharyBiery_CV_2024.pdf
2025-07-16 20:45:12 | app.backend.core.file_processor | INFO | Using .PDF parser for file: data/ZacharyBiery_CV_2024.pdf
2025-07-16 20:45:12 | app.backend.core.file_processor | INFO | Extracting text from 'data/ZacharyBiery_CV_2024.pdf' using local PDF parser (pypdf)
2025-07-16 20:45:13 | app.backend.services.groq | INFO | Running structured query with function calling using ainvoke...
2025-07-16 20:45:14 | app.backend.services.groq | DEBUG | Beautified JSON args:
{
    "candidate_name": "Zachary Biery",
    "certificate": [],
    "comment": "Zachary Biery is a highly skilled and motivated individual with experience in data science, business analytics, and software development. He has a strong educational background in business analytics and has worked on various projects and internships, showcasing his technical skills and soft skills.",
    "education": [{
   

In [12]:
import json

combined_input = {
    "candidate": candidate_response,
    "job": job_response
}
combined_input_str = json.dumps(combined_input)
print(combined_input_str)
print(len(combined_input_str)/4)

{"candidate": {"candidate_name": "Zachary Biery", "certificate": [], "comment": "Zachary Biery is a highly skilled and motivated individual with experience in data science, business analytics, and software development. He has a strong educational background in business analytics and has worked on various projects and internships, showcasing his technical skills and soft skills.", "education": [{"field": "Business Analytics", "institution": "University of Cincinnati", "level": "Masters", "year": "2025"}, {"field": "Business Analytics", "gpa": "4.0", "institution": "University of Cincinnati", "level": "Bachelor's", "year": "2024"}], "email": "bieryzt@mail.uc.edu", "job_recommended": ["Data Scientist", "Business Analyst", "Software Engineer"], "phone_number": "(513)491-2255", "responsibilities": ["Data analysis", "Machine learning model development", "Software development", "Data visualization", "Workflow automation"], "roles": [{"company": "Medpace", "summary": "Trained and implemented N

In [13]:
match_response = await analyzer.match(candidate_analysis=candidate_response, job_analysis=job_response)

2025-07-16 20:46:42 | app.backend.services.groq | INFO | Running structured query with function calling using ainvoke...
2025-07-16 20:46:44 | app.backend.services.groq | DEBUG | Beautified JSON args:
{
    "certificate": {
        "comment": "The job does not require any specific certificates, and the candidate does not have any certificates listed.",
        "score": 0
    },
    "domain": {
        "comment": "The job is in the domain of Artificial Intelligence, while the candidate has experience in data science, business analytics, and software development. There is some relevance but not a direct match.",
        "score": 50
    },
    "education": {
        "comment": "The job requires a Bachelor's or Master's in Computer Science. The candidate has a Master's and Bachelor's in Business Analytics, which is related but not directly aligned with Computer Science.",
        "score": 80
    },
    "experience": {
        "comment": "The job requires 2+ years of experience in applying 

In [14]:
def compute_match_score(match_result: dict) -> float:
    weights = {
        "education": 0.20,
        "experience": 0.25,
        "technical_skill": 0.25,
        "responsibility": 0.20,
        "soft_skill": 0.05,
        "domain": 0.05,
        "certificate": 0,
    }

    total_score = 0
    for key, weight in weights.items():
        score = match_result.get(key, {}).get("score", 0)
        total_score += score * weight

    return round(total_score, 2)

print(compute_match_score(match_response))


70.5


In [1]:
from app.backend.services.factory import AIServiceFactory

service = await AIServiceFactory.create_service()
response = await service.query("What is the area of the UK?")
print(response)

2025-07-07 00:38:58 | app.backend.services.groq | DEBUG | Retrieving Groq API key from Azure Key Vault...
2025-07-07 00:39:01 | app.backend.services.groq | DEBUG | Groq API key retrieved.


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000255A5E7B1C0>


2025-07-07 00:39:01 | app.backend.services.groq | INFO | Calling Groq model 'llama3-70b-8192' using ainvoke...
The total area of the United Kingdom (UK) is approximately 243,610 square kilometers (93,625 square miles). This includes:

* England: 130,279 km² (50,261 sq mi)
* Scotland: 78,772 km² (30,414 sq mi)
* Wales: 20,779 km² (8,023 sq mi)
* Northern Ireland: 14,160 km² (5,470 sq mi)

Note: These figures are approximate and may vary slightly depending on the source.


In [13]:
from datetime import date

prompt = f"""
You are an expert HR recruiter and resume screener. Your task is to extract relevant information from a resume and organize it into structured categories using the format below.

---

**Categories**:

1. **Synopsis** - A personal summary, description, or “about me” section.  
2. **Experience** - Includes jobs/internships under `<job>` and standalone projects under `<project>`.  
3. **Education** - Each degree appears under a `<degreeEntry>` with optional GPA and graduation year.  
4. **Skills** - Divided into technical and non-technical.

---

**Important Instructions**:
- If no synopsis is present, insert the word `Unknown` in the `<synopsis>` tag.
- Do **not** invent or infer missing information. Only extract what is explicitly present.
- Use the exact XML structure and tag names provided.
- All durations must use time units (e.g., `"3 months"`, `"2 years"`).
- Only include UTF-8 characters. Strip bullets, markdown, or decorative formatting.
    - If only a single month is present in the date range, the duration is `"1 month"`
    - For reference, `"Present"` in a job duration refers to {date.today()}

---

**Format your output as follows**:

```xml
<resume>
    <synopsis>
        INSERT SYNOPSIS INFORMATION HERE
    </synopsis>
    <experience>
        <job>
            <company>
                INSERT COMPANY NAME
            </company>
            <duration>
                INSERT JOB DURATION
            </duration>
            <description>
                INSERT JOB DESCRIPTION
            </description>
        </job>
        <!-- Repeat <job> as needed -->

        <project>
            <name>
                INSERT PROJECT NAME
            </name>
            <duration>
                INSERT PROJECT DURATION (if available)
            </duration>
            <description>
                INSERT PROJECT DESCRIPTION
            </description>
        </project>
        <!-- Repeat <project> as needed -->
    </experience>
    <education>
        <degreeEntry>
            <university>
                INSERT UNIVERSITY NAME
            </university>
            <degree>
                <level>
                    INSERT DEGREE LEVEL (e.g., Doctorate, Masters, Baccalaureate)
                </level>
                <name>
                    INSERT DEGREE NAME (e.g., Computer Science)
                </name>
            </degree>
            <graduationYear>
                INSERT GRADUATION YEAR (if available)
            </graduationYear>
            <gpa>
                INSERT GPA (if available)
            </gpa>
        </degreeEntry>
        <!-- Repeat <degreeEntry> as needed -->
    </education>
    <skills>
        <technical>
            <software>
                LIST SOFTWARE SKILLS
            </software>
            <languages>
                LIST PROGRAMMING LANGUAGES
            </languages>
            <frameworks>
                LIST FRAMEWORKS
            </frameworks>
            <tools>
                LIST TECHNICAL TOOLS
            </tools>
        </technical>
        <nontechnical>
            LIST NON-TECHNICAL SKILLS
        </nontechnical>
    </skills>
</resume>

---

You will now be given a resume in plain text. Extract and organize the information as described above.
**Resume**:  
{resume}
"""